In [ ]:
from urllib.request import urlopen
import requests
import concurrent.futures
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def getSoup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup
all_male = []

for idx_page in range(1, 60):
    url_main = (
        f"https://thegioidongho.vip/dong-ho-nam?page={idx_page}"
    )

    content_products_links = (
        getSoup(url_main)
        .find_all(name="h3",class_="product-title")
    )
    
    for h3 in content_products_links:
        link = h3.a["href"]
        # print(link)
        all_male.append(link)


In [10]:
all_female = []

for idx_page in range(1, 23):
    url_main = (
        f"https://thegioidongho.vip/dong-ho-nu?page={idx_page}"
    )

    content_products_links = (
        getSoup(url_main)
        .find_all(name="h3",class_="product-title")
    )
    
    for h3 in content_products_links:
        link = h3.a["href"]
        # print(link)
        all_female.append(link)


527


In [12]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


def get_product_info(url):
    if url is None or url == '':
        raise ValueError('Invalid URL')
        
    soup = get_soup(url)
    
    title_name_elem = soup.select_one("h1.bk-product-name")
    if title_name_elem is None:
        raise ValueError('Cannot find product title')
    title_name = title_name_elem.text.split(" ")
    name = ' '.join(title_name[-2:-1])
    
    price_elem = soup.select_one(".gia.bk-product-price")
    if price_elem is None:
        raise ValueError('Cannot find product price')
    price = price_elem.text.strip()
    
    product_code = title_name[-1]
    details = {}
    table = soup.find("div", {"class": "product-box-content"})
    if table is not None:
        rows = table.find_all('tr')
        for row in rows:
            key_elem, value_elem = row.find_all('td')[:2]  # Lấy 2 td đầu tiên trong hàng
            key = key_elem.text.strip()
            value = value_elem.text.strip()
            details[key.strip()] = value.strip()
    return {"Tên sản phẩm": name, "Giá tiền": price,"Mã sản phẩm":product_code, **details}


def crawl_data_from_web(all_male):
    data_frame_watchs = pd.DataFrame([get_product_info(url) for url in all_male])
    return data_frame_watchs


# url = "https://thegioidongho.vip/dong-ho-nam-chinh-hang-lobinni-l9222-1"

# get_product_info(url)

all_page = [f"https://thegioidongho.vip/dong-ho-nam?page={idx_page}" for idx_page in range(1, 60)]
all_male = [h3.a["href"] for url in all_page for h3 in get_soup(url).select("h3.product-title")]

data_frame_watchs = crawl_data_from_web(all_male)


In [13]:
data_frame_watchs

,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Giới tính,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Kiểu máy,Dành cho,Mã sản phâm
0,LOBINNI,"5,150,000 VNĐ",L9222-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),42 mm,11.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
1,LOBINNI,"5,480,000 VNĐ",L17511-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,10 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
2,LOBINNI,"4,550,000 VNĐ",L12032-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,39 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
3,LOBINNI,"5,280,000 VNĐ",L9010-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
4,LOBINNI,"4,800,000 VNĐ",L16050-2,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,41 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,KASSAW,"5,600,000 VNĐ",K961-8,KASSAW,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),39 mm,10 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,Nhật Bản (Japan Movt),NaN,NaN
1309,KASSAW,"5,600,000 VNĐ",K961-7,KASSAW,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),39 mm,10 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,Nhật Bản (Japan Movt),NaN,NaN
1310,vàng,"15,560,000 VNĐ",K333-2,KASSAW,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),41 mm,11 mm,Thép không gỉ 316L - Mạ vàng 24k,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,Nhật Bản (Japan Movt),NaN,NaN
1311,KASSAW,"3,850,000 VNĐ",K864-3,KASSAW,Thụy Sỹ,Nam,Đồng hồ Automatic,39 mm,10 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,Nhật Bản (Japan Movt),NaN,NaN


In [14]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


def get_product_info(url):
    if url is None or url == '':
        raise ValueError('Invalid URL')
        
    soup = get_soup(url)
    
    title_name_elem = soup.select_one("h1.bk-product-name")
    if title_name_elem is None:
        raise ValueError('Cannot find product title')
    title_name = title_name_elem.text.split(" ")
    name = ' '.join(title_name[-2:-1])
    
    price_elem = soup.select_one(".gia.bk-product-price")
    if price_elem is None:
        raise ValueError('Cannot find product price')
    price = price_elem.text.strip()
    
    product_code = title_name[-1]
    details = {}
    table = soup.find("div", {"class": "product-box-content"})
    if table is not None:
        rows = table.find_all('tr')
        for row in rows:
            key_elem, value_elem = row.find_all('td')[:2]  # Lấy 2 td đầu tiên trong hàng
            key = key_elem.text.strip()
            value = value_elem.text.strip()
            details[key.strip()] = value.strip()
    return {"Tên sản phẩm": name, "Giá tiền": price,"Mã sản phẩm":product_code, **details}


def crawl_data_from_web(all_female):
    data_frame_watchs = pd.DataFrame([get_product_info(url) for url in all_female])
    return data_frame_watchs


# url = "https://thegioidongho.vip/dong-ho-nam-chinh-hang-lobinni-l9222-1"

# get_product_info(url)

all_page = [f"https://thegioidongho.vip/dong-ho-nu?page={idx_page}" for idx_page in range(1, 23)]
all_female = [h3.a["href"] for url in all_page for h3 in get_soup(url).select("h3.product-title")]

data_frame_watchs = crawl_data_from_web(all_female)


In [15]:
data_frame_watchs

,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Kiểu máy,Giới tính,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Dành cho
0,CARNIVAL,"3,950,000 VNĐ",IW573G-1,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
1,CARNIVAL,"3,950,000 VNĐ",IW573G-2,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
2,CARNIVAL,"3,950,000 VNĐ",IW573G-3,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
3,siêu,"7,390,000 VNĐ",IW626G-2,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),34 mm,8.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
4,KASSAW,"5,580,000 VNĐ",K188-1,KASSAW,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),32 mm,11 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
523,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
524,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
525,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam


In [134]:
count_nan = data_frame_watchs.isna().sum()
print(count_nan)

Tên sản phẩm                      0
Giá tiền                          0
Mã sản phẩm                       0
Thương Hiệu                       0
Quốc gia đăng ký thương hiệu      1
Kiểu máy                        339
Giới tính                        78
Loại sản phẩm                     0
Đường kính mặt                    0
Chiều dày                         0
Chất liệu vỏ                      0
Chất liệu dây                     0
Chất liệu mặt kính                0
Độ chịu nước                      0
Nơi sản xuất                      1
Bảo hành                          0
Dành cho                        449
dtype: int64


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        response.raise_for_status()  # raise an exception if the response is not successful
        soup = BeautifulSoup(response.content, "lxml")
    return soup


def get_product_info(url):
    soup = get_soup(url)
    
    title_name_elem = soup.select_one("h1.bk-product-name")
    if not title_name_elem:
        raise ValueError('Cannot find product title')
    title_name = title_name_elem.text.split(" ")
    name = ' '.join(title_name[-2:-1])
    
    price_elem = soup.select_one(".gia.bk-product-price")
    if not price_elem:
        raise ValueError('Cannot find product price')
    price = price_elem.text.strip()
    
    product_code = title_name[-1]
    details = {}
    table = soup.find("div", {"class": "product-box-content"})
    if table:
        rows = table.find_all('tr')
        for row in rows:
            key_elem, value_elem = row.find_all('td')[:2]  # Lấy 2 td đầu tiên trong hàng
            key = key_elem.text.strip()
            value = value_elem.text.strip()
            details[key] = value
    return {"Tên sản phẩm": name, "Giá tiền": price, "Mã sản phẩm": product_code, **details}


def crawl_data_from_web(all_female):
    data_frame_watchs = pd.DataFrame([get_product_info(url) for url in all_female])
    return data_frame_watchs


all_page = [f"https://thegioidongho.vip/dong-ho-nu?page={idx_page}" for idx_page in range(1, 23)]
all_female = [h3.a["href"] for url in all_page for h3 in get_soup(url).select("h3.product-title")]

data_frame_watchs = crawl_data_from_web(all_female)



In [138]:
data_frame_watchs

,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Kiểu máy,Giới tính,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Dành cho
0,CARNIVAL,"3,950,000 VNĐ",IW573G-1,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
1,CARNIVAL,"3,950,000 VNĐ",IW573G-2,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
2,CARNIVAL,"3,950,000 VNĐ",IW573G-3,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),35 mm,12 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
3,siêu,"7,390,000 VNĐ",IW626G-2,IW CARNIVAL,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),34 mm,8.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,24 tháng,NaN
4,KASSAW,"5,580,000 VNĐ",K188-1,KASSAW,Thụy Sỹ,Nhật Bản (Japan Movt),Nữ,Đồng hồ cơ (Automatic),32 mm,11 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
523,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
524,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam
525,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),NaN,NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,Nam


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Hàm tạo và trả về đối tượng BeautifulSoup, được sử dụng để phân tích cú pháp HTML của một trang web
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


# lấy ra tất cả thông tin của các sản phẩm trong trang web thegioidongho
def get_product_info(url):
    if url is None or url == '':
        raise ValueError('Invalid URL')
        
    soup = get_soup(url)
    
    title_name_elem = soup.select_one("h1.bk-product-name")
    if title_name_elem is None:
        raise ValueError('Cannot find product title')
    title_name = title_name_elem.text.split(" ")
    name = ' '.join(title_name[-2:-1])
    
    price_elem = soup.select_one(".gia.bk-product-price")
    if price_elem is None:
        raise ValueError('Cannot find product price')
    price = price_elem.text.strip()
    
    product_code = title_name[-1]
    details = {}

    # tìm phần tử div với class là "product-box-content" trong đối tượng BeautifulSoup
    table = soup.find("div", {"class": "product-box-content"})
    if table is not None:
        #tìm tất cả các thẻ tr (row) trong phần tử div đó và lưu trữ chúng vào biến rows
        rows = table.find_all('tr')
        for row in rows:
            key_elem, value_elem = row.find_all('td')[:2]  # Lấy 2 td đầu tiên trong hàng
            key = key_elem.text.strip()
            value = value_elem.text.strip()
            details[key.strip()] = value.strip()
    return {"Tên sản phẩm": name, "Giá tiền": price,"Mã sản phẩm":product_code, **details}


def crawl_data_from_web(all_male, all_female):
    data_frame_male = pd.DataFrame([get_product_info(url) for url in all_male])
    data_frame_female = pd.DataFrame([get_product_info(url) for url in all_female])
    result = pd.concat([data_frame_male, data_frame_female], ignore_index=True)
    return result


all_page_male = [f"https://thegioidongho.vip/dong-ho-nam?page={idx_page}" for idx_page in range(1,2)]
all_male = [h3.a["href"] for url in all_page_male for h3 in get_soup(url).select("h3.product-title")]

all_page_female = [f"https://thegioidongho.vip/dong-ho-nu?page={idx_page}" for idx_page in range(1, 2)]
all_female = [h3.a["href"] for url in all_page_female for h3 in get_soup(url).select("h3.product-title")]


merged_frame = crawl_data_from_web(all_male, all_female)





In [19]:
num_brands = merged_frame["Thương Hiệu"].nunique()
print("Số lượng thương hiệu có trong cột 'Thương hiệu':", num_brands)


Số lượng thương hiệu có trong cột 'Thương hiệu': 13


In [18]:
merged_frame

,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Giới tính,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Kiểu máy,Dành cho,Mã sản phâm
0,LOBINNI,"5,150,000 VNĐ",L9222-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),42 mm,11.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
1,LOBINNI,"5,480,000 VNĐ",L17511-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,10 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
2,LOBINNI,"4,550,000 VNĐ",L12032-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,39 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
3,LOBINNI,"5,280,000 VNĐ",L9010-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
4,LOBINNI,"4,800,000 VNĐ",L16050-2,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,41 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1836,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1837,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1838,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN


In [19]:
merged_frame.to_csv("thegioidongho.csv")

In [12]:
merged_frame

,Unnamed: 0,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Mã sản phâm,Giới Tính
0,0,LOBINNI,"5,150,000 VNĐ",L9222-1,LOBINNI,Thụy Sỹ,Đồng hồ cơ (Automatic),42 mm,11.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,Nam
1,1,LOBINNI,"5,480,000 VNĐ",L17511-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,42 mm,10 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,Nam
2,2,LOBINNI,"4,550,000 VNĐ",L12032-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,39 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,Nam
3,3,LOBINNI,"5,280,000 VNĐ",L9010-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,42 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,Nam
4,4,LOBINNI,"4,800,000 VNĐ",L16050-2,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,41 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,Nam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,1835,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,
1836,1836,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,
1837,1837,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,
1838,1838,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,


In [6]:
import pandas as pd

# Đọc dữ liệu từ file csv
merged_frame = pd.read_csv("thegioidongho.csv")

# Tạo cột mới chứa thông tin về giới tính và loại khách hàng mà sản phẩm dành cho
merged_frame["Giới Tính"] = merged_frame.apply(
    lambda row: "Nam - " + str(row["Dành cho"]).lower() if row["Giới tính"] == "Nam" 
    else ("Nữ - " + str(row["Dành cho"]).lower() if row["Giới tính"] == "Nữ" else ""), axis=1)

# Xóa 2 cột cũ "Giới tính" và "Dành cho"
merged_frame.drop(["Giới tính", "Dành cho"], axis=1, inplace=True)

# Tách chuỗi để lấy giá trị "Nam"
merged_frame["Giới Tính"] = merged_frame["Giới Tính"].apply(lambda s: s.split(" - ")[0])

# Xóa các dòng có giá trị rỗng hoặc NaN trong cột mới tạo
merged_frame.dropna(subset=["Giới Tính"], inplace=True)

# Lưu dữ liệu vào file csv
merged_frame.to_csv("updated_merged_data.csv", index=False)


In [7]:
merged_frame

,Unnamed: 0,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Kiểu máy,Mã sản phâm,Giới Tính
0,0,LOBINNI,"5,150,000 VNĐ",L9222-1,LOBINNI,Thụy Sỹ,Đồng hồ cơ (Automatic),42 mm,11.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,Nam
1,1,LOBINNI,"5,480,000 VNĐ",L17511-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,42 mm,10 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,Nam
2,2,LOBINNI,"4,550,000 VNĐ",L12032-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,39 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,Nam
3,3,LOBINNI,"5,280,000 VNĐ",L9010-1,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,42 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,Nam
4,4,LOBINNI,"4,800,000 VNĐ",L16050-2,LOBINNI,Thụy Sỹ,Đồng hồ Automatic,41 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,Nam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,1835,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,NaN,
1836,1836,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,NaN,
1837,1837,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,NaN,
1838,1838,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,NaN,


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Hàm tạo và trả về đối tượng BeautifulSoup, được sử dụng để phân tích cú pháp HTML của một trang web
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


# Lấy ra thông tin của một sản phẩm trong trang web thegioidongho
def get_product_info(url):
    soup = get_soup(url)
    
    title_name_elem = soup.find("h1", class_="bk-product-name")
    if title_name_elem is None:
        raise ValueError('Cannot find product title')
    title_name = title_name_elem.text.split(" ")
    name = ' '.join(title_name[-2:-1])
    
    price_elem = soup.find(class_="gia bk-product-price")
    if price_elem is None:
        raise ValueError('Cannot find product price')
    price = price_elem.text.strip()
    
    product_code = title_name[-1]
    details = {}

    # Tìm phần tử div với class là "product-box-content" trong đối tượng BeautifulSoup
    table = soup.find("div", class_="product-box-content")
    if table is not None:
        # Tìm tất cả các thẻ tr (row) trong phần tử div đó và lưu trữ chúng vào biến rows
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')  # Tìm tất cả các thẻ td trong hàng
            if len(cells) >= 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                details[key] = value.strip()
    return {"Tên sản phẩm": name, "Giá tiền": price, "Mã sản phẩm": product_code, **details}


def crawl_data_from_web(all_urls):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(get_product_info, all_urls))
    return pd.DataFrame(results)


all_page_male = [f"https://thegioidongho.vip/dong-ho-nam?page={idx_page}" for idx_page in range(1, 60)]
all_male = [h3.a["href"] for url in all_page_male for h3 in get_soup(url).select("h3.product-title")]

all_page_female = [f"https://thegioidongho.vip/dong-ho-nu?page={idx_page}" for idx_page in range(1, 23)]
all_female = [h3.a["href"] for url in all_page_female for h3 in get_soup(url).select("h3.product-title")]

all_urls = all_male + all_female
merged_frame = crawl_data_from_web(all_urls)


In [24]:
merged_frame.to_csv('../main/raw_data/the_gioi_dong_ho.csv')

,Tên sản phẩm,Giá tiền,Mã sản phẩm,Thương Hiệu,Quốc gia đăng ký thương hiệu,Giới tính,Loại sản phẩm,Đường kính mặt,Chiều dày,Chất liệu vỏ,Chất liệu dây,Chất liệu mặt kính,Độ chịu nước,Nơi sản xuất,Bảo hành,Kiểu máy,Dành cho,Mã sản phâm
0,LOBINNI,"5,150,000 VNĐ",L9222-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ cơ (Automatic),42 mm,11.5 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
1,LOBINNI,"5,480,000 VNĐ",L17511-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,10 mm,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
2,LOBINNI,"4,550,000 VNĐ",L12032-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,39 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
3,LOBINNI,"5,280,000 VNĐ",L9010-1,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,42 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
4,LOBINNI,"4,800,000 VNĐ",L16050-2,LOBINNI,Thụy Sỹ,Nam,Đồng hồ Automatic,41 mm,11 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,5 ATM,Trung Quốc,24 tháng,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,Teintop,"1,550,000 VNĐ",T7004-1,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1836,Teintop,"1,550,000 VNĐ",T7004-4,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1837,Teintop,"1,550,000 VNĐ",T7004-2,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
1838,Teintop,"1,550,000 VNĐ",T7004-6,Teintop,Mỹ (USA),NaN,Đồng hồ điện tử (Pin),27 mm,6 mm,Thép không gỉ 316L,Dây da cao cấp,Sapphire,3 ATM,Trung Quốc,12 tháng,NaN,Nam,NaN
